# Remote Backend Compiler - RBC
*by Pearu Peterson*

## Statement of the problem

*Compile and run user-defined functions in a language-agnostic JIT enabled program semi-remotely.*

### Semi-remote compilation

- *compiler frontend*: user-written source code is parsed and transformed to LLVM IR in client program
- the client program sends the LLVM IR over network to a server program
- *compiler backend*: the server program compiles the LLVM IR to machine code (that can be executed on the server CPU or GPU devices)

### Constraints

- A user-defined function is defined as Python or C/C++ (or any other language with LLVM tools) program
- Client host can run on a 32- or 64-bit OS: Linux, MacOSX, Windows
- Server host runs on 64-bit Linux

## Solution

- *Remote Backend Compiler - RBC*: https://github.com/xnd-project/rbc
- A Python-to-Python prototype uses Numba for LLVM IR generation and Numba llvmlite for machine code compilation
- A Python-to-SQL application uses Numba for LLVM IR generation and OmniSciDB for machine code compilation
  * OmniSciDB is a GPU enabled SQL database server
  * OmniSciDB uses LLVM Compiler C++ library for JIT compilation
  * https://www.omnisci.com/

# Demo of the prototype

1. Start server

  ```python
  import rbc
  rbc.RemoteJIT(host='localhost', port=7890).start_server()
  ```


2. Client example program

In [ ]:
from rbc import RemoteJIT
rjit = RemoteJIT(host='localhost', port=7890)

In [ ]:
# A user-defined function
@rjit('int64(int64, int64)')
def add(a, b):
    return a + b

In [ ]:
print(add)

In [ ]:
add(1, 3)

In [ ]:
add(2, 4)

In [ ]:
try:
    add(1.2, 3.4)
except Exception as msg:
    print(msg)

In [ ]:
add.signature('double(double, double)')
add.signature('complex128(complex128, complex128)');

In [ ]:
add(1, 3.4j)

In [ ]:
add(1.2, 2)

# Demo of the application

Guilherme...